In [ ]:
from pathlib import Path

import Metashape

from mynd.backends import metashape as backend
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result

### Load document into backend

In [ ]:
DOCUMENT_PATH: Path = Path(
    "/data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz"
)

result: Result[str, str] = backend.load_project(DOCUMENT_PATH)
match result:
    case Ok(message):
        logger.info(message)
    case Err(message):
        logger.error(message)

### Define helper functionality to visualize forward and inverse image mapping

In [ ]:
import os

from mynd.collections import CameraGroupIdentifier, StereoCameraGroup
from mynd.backend import metashape as backend
from mynd.geometry import load_hitnet
from mynd.utils.environment import Environment, load_environment

from mynd.tasks.export_stereo_geometry import (
    ExportStereoTask,
    invoke_stereo_export_task,
)

GroupID = CameraGroup.Identifier

def prepare_task_config(
    identifier: GroupID, stereo_camera: StereoCameraGroup
) -> ExportStereoTask.Config:
    """Prepares a stereo export task config by creating setting up paths, creating directories,
    and loading a disparity model."""

    environment: Environment = load_environment().unwrap()

    model: HitnetConfig = load_hitnet(
        environment.resource_directory
        / Path("hitnet_models/hitnet_eth3d_720x1280.onnx")
    ).unwrap()

    # Root directories
    RANGE_DIRECTORY: Path = Path("/data/kingston_snv_01/acfr_stereo_ranges")
    NORMAL_DIRECTORY: Path = Path("/data/kingston_snv_01/acfr_stereo_normals")

    config = ExportStereoTask.Config(
        range_directory=RANGE_DIRECTORY / Path(f"{identifier.label}_ranges"),
        normal_directory=NORMAL_DIRECTORY / Path(f"{identifier.label}_normals"),
        model=model,
        calibrations=stereo_camera.calibrations,
        image_loaders=stereo_camera.image_loaders,
    )

    # Create export directories if they do not exist
    if not config.range_directory.exists():
        os.mkdir(config.range_directory)
    if not config.normal_directory.exists():
        os.mkdir(config.normal_directory)

    return config


def main():
    """Main function."""

    # Since the backend already has a loaded project at this point, 
    # we can start extracting data from it
    identifiers: list[GroupID] = backend.get_group_identifiers()

    target: GroupID = identifiers[0]
    
    match backend.get_stereo_cameras(target)
        case Ok(stereo_camera_groups):
            for stereo_cameras in stereo_camera_groups:
                config: ExportStereoTask.Config = prepare_task_config(
                    target, stereo_cameras
                )
                result: Result[ExportStereoTask.Result, str] = (
                    invoke_stereo_export_task(config)
                )
        case Err(message):
            logger.error(message)


# ---------- Invoke main function ----------
main()